In [7]:
import numpy as np

all values are in mm (except for the output values - those are in cm)

In [8]:
def arr_str(xs):
    if len(xs.shape) == 1:
        return "[" + ", ".join([f'{x:.2f}' for x in xs]) + "]"
    else:
        res = []
        res.append('[')
        for i in range(xs.shape[0]):
            res.append(arr_str(xs[i]))
            res.append(',\n')
        res.append(']')
        return ''.join(res)

In [9]:
phantom_dimensions = np.array([150, 130, 13], dtype=float)
# phantom_closest_vertex = np.array([335, 65, 0], dtype=float)
phantom_closest_vertex = np.array([50, 50, 50], dtype=float)
phantom_scale = 150

phantom_max_dim = np.max(phantom_dimensions)
phantom_normalised_spans = phantom_dimensions / phantom_max_dim / 2
phantom_scaled_spans = phantom_normalised_spans * phantom_scale

phantom_difftactile_position = phantom_closest_vertex + phantom_scaled_spans

phantom_centre_top = phantom_closest_vertex.copy()
phantom_centre_top += np.array([
    phantom_dimensions[0] / 2,
    phantom_dimensions[1] / 2,
    phantom_dimensions[2],
], dtype=float)

sensor_outer_radius = 33
sensor_dome_tip = np.array([
    phantom_centre_top[0],
    phantom_centre_top[1],
    phantom_centre_top[2] + 3.0,
], dtype=float)
sensor_sphere_centre = np.array([
    sensor_dome_tip[0],
    sensor_dome_tip[1],
    sensor_dome_tip[2] + sensor_outer_radius,
])

phantom_diagonal = np.array([75, -65, 0], dtype=float)
phantom_diagonal /= np.linalg.norm(phantom_diagonal)
camera_distance = 20.0
phantom_diagonal *= camera_distance
camera_position = phantom_centre_top + phantom_diagonal
camera_position = np.array([
    camera_position[0],
    camera_position[1],
    camera_position[2] + 30.0,
])

phantom_selected_vertices = np.zeros(shape=(4, 3))
phantom_selected_vertices[0] = phantom_closest_vertex
for i in range(3):
    vertex = np.copy(phantom_closest_vertex)
    vertex[i] += phantom_dimensions[i]
    phantom_selected_vertices[i+1] = vertex

print('difftactile coordinates')
print(f'phantom centre of mass: {arr_str(phantom_difftactile_position / 10)}')
print(f'sensor centre: {arr_str(sensor_sphere_centre / 10)}')
print(f'sensor dome tip: {arr_str(sensor_dome_tip / 10)}')
print(f'phantom centre, top surface (camera lookat): {arr_str(phantom_centre_top / 10)}')
print(f'camera_position: {arr_str(camera_position / 10)}')
print(f'phantom_selected_vertices:\n{arr_str(phantom_selected_vertices / 10)}')



difftactile coordinates
phantom centre of mass: [12.50, 11.50, 5.65]
sensor centre: [12.50, 11.50, 9.90]
sensor dome tip: [12.50, 11.50, 6.60]
phantom centre, top surface (camera lookat): [12.50, 11.50, 6.30]
camera_position: [14.01, 10.19, 9.30]
phantom_selected_vertices:
[[5.00, 5.00, 5.00],
[20.00, 5.00, 5.00],
[5.00, 18.00, 5.00],
[5.00, 5.00, 6.30],
]


In [10]:
def global_to_local_velocity(global_velocity, global_transformation_matrix):
    """
    Transforms a 3D velocity vector from the global coordinate frame
    to the local coordinate frame.

    Args:
        global_velocity (np.ndarray): A 3-element numpy array representing
                                       the velocity vector in the global frame (a, b, c).
        global_transformation_matrix (np.ndarray): A 4x4 numpy array representing
                                                   the global transformation matrix.

    Returns:
        np.ndarray: A 3-element numpy array representing the velocity vector
                    in the local coordinate frame.
    """
    if global_velocity.shape != (3,):
        raise ValueError("global_velocity must be a 3-element numpy array.")
    if global_transformation_matrix.shape != (4, 4):
        raise ValueError("global_transformation_matrix must be a 4x4 numpy array.")

    # Extract the 3x3 rotation matrix from the global transformation matrix
    R_global_to_local = global_transformation_matrix[:3, :3]

    # The inverse of a rotation matrix is its transpose
    R_local_to_global = R_global_to_local.T

    # Transform the global velocity vector to the local frame
    local_velocity = np.dot(R_local_to_global, global_velocity)

    return local_velocity

In [11]:
trans_mat = np.array([
    [1, 0, 0, 7.5],
    [0, 0, 1, 6.5],
    [0, -1, 0, 5.4],
    [0, 0, 0, 1],
], dtype=float)

velocity_global = np.array([0, 0, -10], dtype=float)
velocity_local = global_to_local_velocity(velocity_global, trans_mat)
print(velocity_local)

[ 0. 10.  0.]
